In [1]:
import torchvision

In [2]:
!pwd

/content


In [3]:

import zipfile

# Ruta del archivo ZIP
zip_file_path = '/content/tesis-main.zip'

# Ruta donde quieres extraer los archivos
extract_to_path = '/content/'

# Descomprimir el archivo
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print("Archivos extraídos con éxito.")

Archivos extraídos con éxito.


In [2]:
import sys
sys.path.append('/content/tesis-main/')


In [5]:

import drive as drive_downloader
drive_downloader.download_files('14_UUqgoWDPPNwXvdFjDoJomHTu5l85l6')

Files inside the folder , [{'mimeType': 'application/vnd.google-apps.spreadsheet', 'id': '1SHWH9O5cULOyPZDuWJbTE0HfeDqC9cBQqTc_CS6IvdM', 'name': 'Casos'}, {'mimeType': 'application/vnd.google-apps.folder', 'id': '1dfqDJRa6duhIDf__utSxk11wMpqg1fUh', 'name': '13 - Airbnb'}, {'mimeType': 'application/vnd.google-apps.folder', 'id': '10q4CWNyyK94p3hVB0mSLHakZTwvJsRPz', 'name': '12 - PedidosYa'}, {'mimeType': 'application/vnd.google-apps.folder', 'id': '1j_pdSVERiTsFvWFIXRACnR6uLMU5xLM2', 'name': '11 - Wikipedia'}, {'mimeType': 'application/vnd.google-apps.folder', 'id': '14-LkHSI30EJz5aiYfGiMoxD_JDIS8HpD', 'name': '10 - Alibaba.com'}, {'mimeType': 'application/vnd.google-apps.folder', 'id': '1vXuHt3dkYm-gfi7ZKIx83CtDdzA10f7v', 'name': '07 - CNN'}, {'mimeType': 'application/vnd.google-apps.folder', 'id': '1XZDsKeluRAluJTfQpB9lIk-okA6nZSeb', 'name': '09 - iOS - Burguerking'}, {'mimeType': 'application/vnd.google-apps.folder', 'id': '1ISGW3R1If7I6LvRjQAYK6xfaUnYVcDh0', 'name': '08 - Play Store

Files inside the folder , [{'mimeType': 'application/json', 'id': '1_YZBfkaQa1gsMg-PDFHC3Y3_PTOgHSpQ', 'name': 'Airbnb.json'}, {'mimeType': 'video/mp4', 'id': '1iATtQEYrVO1XQF7uJnWqUrUViVkfj7R3', 'name': 'Airbnb.mp4'}]


Descargando Airbnb.json 100%.
Descargando Airbnb.mp4 41%.
Descargando Airbnb.mp4 83%.
Descargando Airbnb.mp4 100%.


Files inside the folder , [{'mimeType': 'application/json', 'id': '1ENDBW1ruaT03y6iGPKWYjYzS08XVCG6J', 'name': 'pedidosya.json'}, {'mimeType': 'video/mp4', 'id': '1J_l9SFqBIBP0G2AhEOXiAKVJVfuSWVkn', 'name': 'pedidosya.mp4'}]


Descargando pedidosya.json 100%.


KeyboardInterrupt: ignored

* Carga el video utilizando cv2.VideoCapture.
* Extrae los frames del video uno por uno.
* Preprocesa cada frame para que sean compatibles con el modelo de PyTorch (tamaño, normalización, etc.).
* Usa un modelo preentrenado de PyTorch (como ResNet) para extraer características de cada frame.
* Aplica un algoritmo de clustering (como K-Means) a las características extraídas.
* Asigna cada frame al cluster correspondiente.
* Visualiza los resultados mostrando los frames y su cluster correspondiente.

In [3]:
import numpy as np

class SimilarityClustering:
    def __init__(self, threshold):
        self.threshold = threshold
        self.labels_ = None

    def fit(self, features):
        cosine_similarities = np.array([cosine_similarity(features[i], features[i + 1])
                                        for i in range(len(features) - 1)])

        cosine_similarities = np.insert(cosine_similarities, 0, cosine_similarity(features[0], features[0]))

        self.labels_ = label_clusters(cosine_similarities, self.threshold)

cosine_similarity = lambda vec1, vec2: np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def label_clusters(cosine_similarities, threshold):
    clusters = np.zeros(len(cosine_similarities), dtype=int)
    current_cluster = 0
    for i in range(1, len(cosine_similarities)):
        #print(i,cosine_similarities[i])

        if cosine_similarities[i] < threshold:
            current_cluster += 1
        clusters[i] = current_cluster
    #print(clusters)
    return clusters

def clustering_function(model):
    def get_labels(features):
        model.fit(features)
        labels = model.labels_
        return labels
    return get_labels


In [ ]:
import torch
from sklearn.cluster import AgglomerativeClustering
import warnings
from models.resnet18 import get_model as get_resnet18
from preprocessing.transforms import BASELINE
from clustering.metrics import eval_massive_cluster
from clustering.model import clustering_function
from experiment_framework import experiment
from validation import VALIDATION_DATASET
from utils import load_dataset
warnings.filterwarnings('ignore')
import wandb

torch.manual_seed(47)
import random
random.seed(47)
import numpy as np
np.random.seed(47)

if torch.cuda.is_available():
 dev = "cuda:0"
else:
 dev = "cpu"


device = torch.device(dev)
epochs = 10
steps=1
clustering_eval_function = lambda cluster_labels, tags: eval_massive_cluster(cluster_labels, tags, steps, epochs)

dataset = load_dataset(VALIDATION_DATASET.keys())

wandb.login()

In [13]:


DEFAULT_PARAMS = {
      "dataset": dataset,
      "evaluation_function": clustering_eval_function,
      "show": True,
      "device": device
   }


EXPERIMENT_DEFINITIONS= [
    {
        'name': 'ResnetAglomerative',
        'model': get_resnet18(device),
        'grouper_function': clustering_function(AgglomerativeClustering(None, distance_threshold=50)),
        'preprocessing': BASELINE,
    },
    {
        'name': 'SimilarityClustering',
        'model': get_resnet18(device),
        "grouper_function": clustering_function(SimilarityClustering(threshold=0.999)),
        'preprocessing': BASELINE,
    }
]

for exp_definition in EXPERIMENT_DEFINITIONS:
  params = DEFAULT_PARAMS.copy()
  params.update(exp_definition)
  run = wandb.init(project="ort-tesis",group=exp_definition['name'],config=exp_definition)

  result = experiment(**params)
  run.log(result._asdict())
  run.finish()


Experiment name: ResnetAglomerative Airbnb.mp4
Precision - Mean: 0.62, std: 0.0357667737
Recall - Mean: 0.50, std: 0.0402837613
F1-Score - Mean: 0.51, std: 0.0375960712
Accuracy - Mean: 0.50, std: 0.0402837613

Experiment name: ResnetAglomerative AVG
Precision - Mean: 0.62, std: 0.0000000000
Recall - Mean: 0.50, std: 0.0000000000
F1-Score - Mean: 0.51, std: 0.0000000000
Accuracy - Mean: 0.50, std: 0.0000000000



Experiment name: SimilarityClustering Airbnb.mp4
Precision - Mean: 0.94, std: 0.0000000000
Recall - Mean: 0.49, std: 0.0000000000
F1-Score - Mean: 0.61, std: 0.0000000000
Accuracy - Mean: 0.49, std: 0.0000000000

Experiment name: SimilarityClustering AVG
Precision - Mean: 0.94, std: 0.0000000000
Recall - Mean: 0.49, std: 0.0000000000
F1-Score - Mean: 0.61, std: 0.0000000000
Accuracy - Mean: 0.49, std: 0.0000000000


In [8]:
!pip install wandb
import wandb
from wandb.keras import WandbCallback

wandb.login()





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.0 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True